In [31]:
import numpy as np
from numpy.fft import fft

In [32]:
folder = 'data_N8b4d'
outfile = 'simulation_params_N8b4d.dat'
maxN = 4501

In [33]:
#Read earnings sequences
earn_data = []
for n in range(0,maxN,5):
    earnings = []
    with open(folder+"/earnings_evol_{0:04d}.dat".format(n), "r") as fdata:
        for line in fdata:
            v = list(map(int,line.split()))
            earnings.append(v[1:])
    earn_data.append(earnings)
earn_data = np.array(earn_data)
earn_data.shape

(901, 501, 8)

### Efficiency
Average utility per agent over the last $\tau$ rounds of the $m$th simulation.
$$E_m=\frac{1}{\tau}\sum_{t=T-\tau}^{T}\frac{1}{N}\sum_{i=1}^NU_{i,m}(t)=\frac{1}{\tau}\sum_{t=T-\tau}^{T}\bar{U}_{m}(t),$$
where $N$ is the number of agents, and $T$ is the length of a simulation measured in rounds. $U_{i,m}(t)$ is the utility of agent $i$ in simulation $m$ at round $t$, scaled each round as
$$U_i=\frac{u_i}{N\cdot|u|_{max}},$$
where $u_i$ is the plain utility of agent $i$, $|\cdot|_{max}$ is the maximum absolute $u_i$ value for the round in a particular simulation. With this choice of normalization the utility of a single agent is bounded by $-1/N\leq U_i\leq 1/N$, and the total utility per round is bounded between -1 and 1. The downside is that it is not possible to compare for the net utilities of different simulations. If this is needed, it is better only to normalize by $N$.

### Inequality
Average standard deviation of the utility per agent over the last $\tau$ rounds of the $m$th simulation.
$$Ieq_m=\frac{1}{\tau}\sum_{t=T-\tau}^{T}\sqrt{\frac{1}{N-1}\sum_{i=1}^N(U_{i,m}(t)-\bar{U}_m(t))^2}=\frac{1}{\tau}\sum_{t=T-\tau}^{T}\sigma_m(t).$$

In [34]:
def unorm(row):
    mx = np.abs(row).max()
    N = len(row)
    if mx != 0:
        return np.array([e/(N*mx) for e in row])
    else:
        return row

def nnorm(row):
    N = len(row)
    return np.array([e/N for e in row])

In [35]:
# Scale utility by N
utilities = np.array([[nnorm(e) for e in earnings] for earnings in earn_data])
utilities.shape

(901, 501, 8)

In [36]:
# Scale utility full
n_utilities = np.array([[unorm(e) for e in earnings] for earnings in earn_data])
n_utilities.shape

(901, 501, 8)

In [37]:
tau = 100
N = utilities.shape[2]

In [38]:
# Slice the last tau steps
utilities = utilities[:,-tau:,:]
utilities.shape

(901, 100, 8)

In [39]:
# Slice the last tau steps
n_utilities = n_utilities[:,-tau:,:]
n_utilities.shape

(901, 100, 8)

In [40]:
#Efficiencies
efficiencies = [np.array([u.mean() for u in util]).mean() for util in utilities]
#efficiencies

In [41]:
#Inequalities
inequalities = [np.array([u.std() for u in util]).mean() for util in utilities]
#inequalities

In [42]:
n_efficiencies = [np.array([u.mean() for u in util]).mean() for util in n_utilities]

In [43]:
n_inequalities = [np.array([u.std() for u in util]).mean() for util in n_utilities]

### Entropy
Entropy per agent of the sequence of states of the system during simulation $m$
$$h_m=-\frac{1}{N}\sum_kp_{k,m}\log_2p_{k,m},$$
where $p_{k,m}$ is the probability of finding state $1\leq k\leq 2^N$ in the sequence of length $\tau$ at simulation $m$
$$p_{k,m}=\frac{n_{k,m}}{\tau},$$
where $n_{k,m}$ is the number of occurences of state $k$ in the sequence of simulation $m$. The maximum entropy per agent for a system with $N$ agents is $\log_2(2^N)/N=1$.

In [44]:
def bin_to_dec(b_str):
    if len(b_str) > 1:
        return int(b_str[0])*2**(len(b_str)-1) + bin_to_dec(b_str[1:])
    else:
        return int(b_str[0])

In [45]:
#Read state sequences
state_bins = []
state_data = []
for n in range(0,maxN,5):
    bins = []
    states = []
    with open(folder+"/state_evol_{0:04d}.dat".format(n), "r") as fdata:
        for line in fdata:
            v = line.split()
            bins.append(v[1])
            states.append(bin_to_dec(v[1]))
    state_bins.append(bins)
    state_data.append(states)
state_bins = np.array(state_bins)
state_data = np.array(state_data)
state_data.shape

(901, 501)

In [46]:
#Slice the last tau steps
state_data = state_data[:,-tau:]
state_data.shape

(901, 100)

In [47]:
#Compute the distributions
def comp_dist(sequence, N):
    dist = np.zeros(2**N)
    u, counts = np.unique(sequence, return_counts=True)
    for i in range(len(u)):
        dist[u[i]] = counts[i]
    return dist/dist.sum()

#Compute the entropy per agent
def entropy(distribution):
    N = np.log2(len(distribution))
    h = 0
    for p in distribution:
        if p > 0:
            h -= p * np.log2(p)
    return h/N

In [48]:
#Entropies
entropies = [entropy(comp_dist(states, N)) for states in state_data]
#entropies

### Frequency
Frequency with wich a state pattern repeats in the time series. The frequency is computed as the inverse of the period thet the pattern presents.

In [49]:
##The fft is not getting the correct frequencies in several cases, in particular is  period>3
#def get_highest_freq(series):
#    X = fft(series)
#    sr = 1     #sampling rate (1 state every time step)
#    N = len(X)
#    n = np.arange(N)
#    T = N/sr
#   freq = n/T
#   return freq[list(np.abs(X[1:])).index(max(np.abs(X[1:int(N/2)])))]

def get_period(series):
    val = series[1]
    count = 1
    for i in range(2,len(series)):
        if series[i] == val:
            break
        count += 1
    return count

In [50]:
#Frequencies
frequencies = [1/get_period(states) for states in state_data]
#frequencies

In [51]:
#for i in range(len(frequencies)):
#    if frequencies[i] == 0:
#       print(state_data[i])

### Information per agent
Every agent develops a strategy to choose its next action given its knowledge of the previous state. This strategy is stored in its transition matrix an utility function. However, some strategies might be simple and depend on little to no knowledge as, for example, the strategy to follow always the same action irrespective of the previous state. Other strategies might be more complex, and depend on the knowledge of a finite number of bits from the previous state.</b>

The information per agent of a particular simulation refers to the average number of bits per agent necessary to follow the strategy the agents have adopted.

In [52]:
neighbors = []
for n in range(0,maxN,5):
    neigh = []
    with open(folder+"/known_idx_{0:04d}.dat".format(n), "r") as fdata:
        for line in fdata:
            v = list(map(int,line.split()))
            neigh.append(v[1:])
    neighbors.append(neigh)
neighbors = np.array(neighbors)
neighbors.shape

(901, 8, 4)

In [53]:
def get_pattern(data):
    p = get_period(data)
    pat = data[-p:]
    pattern = {}
    for i in range(len(pat[0])):
        pattern[i] = [pat[j][i] for j in range(len(pat))]
    return pattern

def filter_pattern(pattern, f):
    f_pat = {}
    for i in pattern.keys():
        if f[i]:
            f_pat[i] = pattern[i]
    return f_pat

def combinations(n, k):
    combos = []
    if (k == 0):
        return [[]]
    elif (k == 1):
        return [[i] for i in n] 
    for i in range(len(n)): 
        head = n[i:i+1]

        tail = combinations(n[i+1:],k-1)

        for j in range(len(tail)):
            #print("tail[j]", tail[j])
            if (type(tail[j]) == int):
                combo = head + [tail[j]]
            else:
                combo = head + tail[j]
            combos.append(combo)   
    return combos

def get_strategy(pattern, idx, known_idx):
    my_neighb = np.delete(known_idx[idx],np.where(known_idx[idx]==idx)[0]).tolist()
    N = len(pattern)
    l = len(pattern[idx])
    #print(my_neighb)

    for Nneigh in range(0,len(my_neighb)+1):
        neighb_lists = combinations(my_neighb, Nneigh)
        neighb_lists = [[idx] + nl for nl in neighb_lists]
        #print(neighb_lists)
        
        for n in neighb_lists:
            columns = sorted(n)
            mask = [(i in columns) for i in range(N)]
            f_pat = filter_pattern(pattern, mask)
            
            #print(f_pat)
            strat = {}
            test = True
            for i in range(l):
                p = ''.join(f_pat[a][i] for a in columns)
                #print(p)
                if p not in strat:
                    strat[p] = f_pat[idx][(i+1)%l]
                else:
                    if strat[p] != f_pat[idx][(i+1)%l]:
                        test = False
                        break
                test = True
        
            if test:   
                return tuple(columns), strat
    return None



In [54]:
#for NNN in range(600):
#    pat = get_pattern(state_bins[NNN])
#    print(pat)
#    print(neighbors[NNN])
#    strat = get_strategy(pat, 0, neighbors[NNN])
#    print(strat)
#    min_bits = len(strat[0]) if len(strat[1]) > 1 else 0
#    print(min_bits)
#    print()

In [55]:
def av_min_info(state, neighbors):
    av = 0
    pattern = get_pattern(state)
    for i in pattern.keys():
        strat = get_strategy(pattern,i,neighbors)
        av += len(strat[0]) if len(strat[1]) > 1 else 0
    return av/len(pattern)

In [56]:
#Information per agent
info_per_agent = [av_min_info(state_bins[i],neighbors[i]) for i in range(len(state_bins))]
#info_per_agent

### Inter-agent entropy
In a single round of a simulation, measures the entropy of the binary states of the agents.

In [57]:
#Compute the entropy per agent
def entropy_i(bin_state):
    N = len(bin_state[0])
    H_ac = 0
    for s in bin_state[1:]:
        p_zeros = s.count('0')/N
        p_ones = s.count('1')/N
        if p_zeros != 0 and p_ones != 0:
            H_ac += -p_zeros*np.log2(p_zeros)-p_ones*np.log2(p_ones)
    return H_ac/len(bin_state)

In [58]:
iagent_entropy = [entropy_i(s) for s in state_bins]
#iagent_entropy

### Save parameters

In [59]:
with open(outfile, "w") as fout:
    for i in range(state_data.shape[0]):
        fout.write("{0} {1} {2} {3} {4} {5} {6} {7} {8}\n".format(i,efficiencies[i],inequalities[i],n_efficiencies[i],
                                                                  n_inequalities[i],entropies[i],frequencies[i],
                                                                  info_per_agent[i],iagent_entropy[i]))

In [60]:
#for i in range(len(efficiencies)):
#    if inequalities[i] < 1.e-3:
#        print(state_data[i])
#        print(utilities[i])